# Opérateurs d'agrégation

les opérations d'agrégations permettent de:
- Rassembler les valeurs de plusieurs documents ensemble.
- Effectuer des opérations sur plusieurs documents et returner une seul valeur.


les agrégateurs peuvent-etre utilisé dans une pipeline.

In [1]:
from pymongo import MongoClient
from pandas import DataFrame as df
import dotenv
import os

def connect_to_db(url=None):
    return MongoClient(url)

dotenv.load_dotenv()

connection = connect_to_db(os.getenv("MONGO_URI"))
db = connection["update_operation"]
collection = db["aggregators"]

def display_collection(collection):
    display(df(collection).drop("_id", axis=1))

In [2]:
# Données de tests

def create_people(name, profession, people_saved, power):
    return ({
        "name": name,
        "job": profession,
        "people_saved": people_saved,
        "power": power,
    })

collection.delete_many({}) 

collection.insert_many([
    create_people("Superman", "Hero", 3_000_000, ["Oui"]),
    create_people("Batman", "Hero", 15_994_000, ["Money", "Arts martiaux", "Technology"]),
    create_people("Mr Terrific", "Hero", 302_002, ["Cerveau", "Technology"]),
    create_people("Silver Surfer", "Heraut", 16_000_000_000, ["Silver", "Surfer"]),
    create_people("Docteur Doom", "Villain", 0, []),
    create_people("Posion Ivy", "Villain", -560, ["Nature"]),
    create_people("Jean Michel", "Comptable", 1, ["Mathématiques"]),
])

display_collection(collection.find())

,name,job,people_saved,power
0,Superman,Hero,3000000,[Oui]
1,Batman,Hero,15994000,"[Money, Arts martiaux, Technology]"
2,Mr Terrific,Hero,302002,"[Cerveau, Technology]"
3,Silver Surfer,Heraut,16000000000,"[Silver, Surfer]"
4,Docteur Doom,Villain,0,[]
5,Posion Ivy,Villain,-560,[Nature]
6,Jean Michel,Comptable,1,[Mathématiques]


## `$match`

Filtre les documents avec le prédicat de requetes spécifié.  
Un prédicat de requete est une expression qui retourne un booléen. La syntax du prédicat est la meme que pour un requete `find()`. Les opérateurs de comparaisons peuvent donc etre utilisé.

In [7]:
# On souhaite recupérer seulement les personnes dans les professions sont 'Hero'
res = collection.aggregate([{ '$match': {'job': {'$eq': 'Hero'}} }])
print("Toutes les personnes ayant pour profession 'Hero':")
display_collection(res)


Toutes les personnes ayant pour profession 'Hero':


,name,job,people_saved,power
0,Superman,Hero,Beaucoup,[Oui]
1,Batman,Hero,15999994,"[Money, Arts martiaux, Technology]"
2,Mr Terrific,Hero,302002,"[Cerveau, Technology]"


## `$group`
Combine plusieurs documents en fonction d'un champs défini.  
On peut utiliser des operateurs d'accumulation tel que `min`, `max`, `count`, `avg`, `first`, `last` et bien d'autres. La liste complète est disponible [ici](https://www.mongodb.com/docs/manual/reference/operator/aggregation/group/)

In [8]:
# On souhaite récupérer une liste de toutes les personnes en fonction de leur métier.
res = collection.aggregate([
    { "$group": {
        "_id": "$profession",
        "job": {"$first": "$job"},
        "names": {"$push": "$name"},
    }}
])
print("Liste des personnes en fonction de leur métier:")
display_collection(res)

Liste des personnes en fonction de leur métier:


,job,names
0,Hero,"[Superman, Batman, Mr Terrific, Silver Surfer,..."


## `$project`
Crée un nouveau document, à partir des documents entrants.
Permet de créer de nouveau champs.  
Permet d'inclure ou d'exclure des champs d'un document/requete.

In [ ]:
res = collection.aggregate([{
    "$project": {
        "power": 1,
        "people_saved": 1,
        "secret_name": { "$replaceAll": 
            {"input": "$name",  "find": "a", "replacement": "e"}
        },
    }},
    {"$project": { "power": 0 }}, 
])
print("Liste des noms des personnes :")
display_collection(res)

Liste des noms des personnes :


,people_saved,secret_name
0,3000000,Supermen
1,15994000,Betmen
2,302002,Mr Terrific
3,16000000000,Silver Surfer
4,0,Docteur Doom
5,-560,Posion Ivy
6,1,Jeen Michel


## `$sort`
Trie les documents selon le champs spécifié

In [10]:
res = collection.aggregate([{"$sort": {"people_saved": 1}}])
print("Personne ayant sauvé le plus de personnes par ordre croissant :")
display_collection(res)

res = collection.aggregate([{"$sort": {"name": -1}}])
print("Nom des personnes par ordre alphabétique inversé :")
display_collection(res)

Personne ayant sauvé le plus de personnes par ordre croissant :


,name,job,people_saved,power
0,Posion Ivy,Villain,-560,[Nature]
1,Docteur Doom,Villain,0,[]
2,Jean Michel,Comptable,1,[Mathématiques]
3,Mr Terrific,Hero,302002,"[Cerveau, Technology]"
4,Batman,Hero,15999994,"[Money, Arts martiaux, Technology]"
5,Silver Surfer,Heraut,16000000000,"[Silver, Surfer]"
6,Superman,Hero,Beaucoup,[Oui]


Nom des personnes par ordre alphabétique inversé :


,name,job,people_saved,power
0,Superman,Hero,Beaucoup,[Oui]
1,Silver Surfer,Heraut,16000000000,"[Silver, Surfer]"
2,Posion Ivy,Villain,-560,[Nature]
3,Mr Terrific,Hero,302002,"[Cerveau, Technology]"
4,Jean Michel,Comptable,1,[Mathématiques]
5,Docteur Doom,Villain,0,[]
6,Batman,Hero,15999994,"[Money, Arts martiaux, Technology]"


## `$limit`
Limite le nombre de documents retours.

In [11]:
res = collection.aggregate([
    {"$sort": {"people_saved": -1}},
    {"$limit": 3}
])
print("Les 3 personnes qui ont sauvé le plus de monde")
display_collection(res)

Les 3 personnes qui ont sauvé le plus de monde


,name,job,people_saved,power
0,Superman,Hero,Beaucoup,[Oui]
1,Silver Surfer,Heraut,16000000000,"[Silver, Surfer]"
2,Batman,Hero,15999994,"[Money, Arts martiaux, Technology]"


## `$skip`
Saute un nombre d'élément défini.

In [12]:
res = collection.aggregate([
    {"$sort": {"people_saved": -1}},
    {"$skip": 3}
])
print("Le reste du leaderboard")
display_collection(res)

Le reste du leaderboard


,name,job,people_saved,power
0,Mr Terrific,Hero,302002,"[Cerveau, Technology]"
1,Jean Michel,Comptable,1,[Mathématiques]
2,Docteur Doom,Villain,0,[]
3,Posion Ivy,Villain,-560,[Nature]


## `$count`
Compte le nombre de document.

In [13]:
res = collection.aggregate([
    {"$match": {"job": {"$eq": "Hero"}}},
    {"$count": "hero_count"}
])
print(f'Nombre de héros: {res.to_list()[0]['hero_count']}')

Nombre de héros: 3


## `$lookup`
Union ensembliste ou jointure externe

In [14]:
operations = db['operations']
operations.insert_many([
    {'superhero': 'Superman', 'name': 'Invasion extra-terrestre','location': 'Metropolis, USA'},
    {'superhero': 'Batman', 'name': "Evasion d'Asylum", 'location': 'Arkham city, USA'}
])

res = collection.aggregate([
    { "$lookup": {
        "from": "operations",
        "localField": "name",
        "foreignField": "superhero",
        "as": "operations",
        "pipeline": [{
            "$project" : {"_id": 0}
        }]
    }},
    { "$match" : { "operations" : {"$not" : { "$size": 0}}}}
])
print("les operations menés par les heros")
display_collection(res)

les operations menés par les heros


,name,job,people_saved,power,operations
0,Superman,Hero,Beaucoup,[Oui],"[{'superhero': 'Superman', 'name': 'Invasion e..."
1,Batman,Hero,15999994,"[Money, Arts martiaux, Technology]","[{'superhero': 'Batman', 'name': 'Evasion d'As..."


## `$unwind`
Deroule un array et crée un document pour chaque valeur déroulé.

In [15]:
res = collection.aggregate([
    {"$unwind": "$power"},
    {"$group": {
        "_id": "$power",
        "power": {"$first": "$power"}
    }},
    {"$sort": {"power": 1}}
])
print("tous les pouvoirs par ordre alphabétiques:")
display_collection(res)

tous les pouvoirs par ordre alphabétiques:


,power
0,Arts martiaux
1,Cerveau
2,Mathématiques
3,Money
4,Nature
5,Oui
6,Silver
7,Surfer
8,Technology
